In [1]:
import pandas as pd
data_path = "../../data/open_source_8454_combine_short_description.csv"
df = pd.read_csv(data_path)
# df_label_0 = df[df["Label"] == 0]
# df_label_0_to_drop = df_label_0.sample(n=3000, random_state=42)
# df = df.drop(df_label_0_to_drop.index)
df

,Inbound Message,Label
0,login issue verified user details employee man...,0
1,outlook hello team meetings skype meetings etc...,0
2,cant log vpn cannot log vpn best,0
3,unable access tool page unable access tool page,0
4,skype error skype error,0
...,...,...
8449,emails coming mail good afternoon receiving em...,22
8450,telephony software issue telephony software issue,0
8451,vip windows password reset tifpdchb pedxruyf v...,0
8452,machine est funcionando unable access machine ...,44


In [6]:
label_counts = df['Label'].value_counts().sort_values(ascending=False)
print(label_counts)

Label
0     3974
47     661
17     289
4      257
48     252
12     241
11     215
23     200
42     184
5      145
2      140
39     129
6      118
18     116
26     107
31     100
22      97
10      88
49      87
8       85
9       81
46      68
25      63
27      61
19      56
32      45
21      44
33      40
7       39
34      37
13      36
36      35
1       31
15      31
3       30
14      29
37      27
44      25
16      25
43      20
24      20
30      19
20      18
29      16
35      15
28      15
40      14
41      11
45      11
38       7
Name: count, dtype: int64


In [7]:
mode = label_counts.mode().values[0]             # 眾數（出現次數最多的 label 數量）
mean = label_counts.mean()                       # 平均數
median = label_counts.median()                   # 中位數
std_dev = label_counts.std()                     # 標準差
min_count = label_counts.min()                   # 最小值
max_count = label_counts.max()                   # 最大值
print(f"Label 出現次數的統計資訊：")
print(f"總類別數：{len(label_counts)}")
print(f"最多出現次數（眾數）：{mode}")
print(f"平均出現次數：{mean:.2f}")
print(f"中位數：{median}")
print(f"標準差：{std_dev:.2f}")
print(f"最少出現次數：{min_count}")
print(f"最多出現次數：{max_count}")

Label 出現次數的統計資訊：
總類別數：50
最多出現次數（眾數）：11
平均出現次數：169.08
中位數：50.5
標準差：560.08
最少出現次數：7
最多出現次數：3974


In [3]:
from gensim.models import Word2Vec
import random

# 讀取資料集
raw_data = df

# 處理文本，將每個文本轉換為詞彙列表
corpus = [text.split() for text in raw_data['Inbound Message']]

# 訓練Word2Vec模型
model = Word2Vec(corpus, vector_size=100, window=5, min_count=1, sg=0)

random_words = 3

# 定義替換詞函數，隨機挑選random_words個詞並替換
def replace_random_words(text, model, num_replacements=random_words):
    words = text.split()
    replaced_text = []
    replaced_indices = random.sample(range(len(words)), min(num_replacements, len(words)))
    for i, word in enumerate(words):
        if i in replaced_indices:
            similar_words = model.wv.most_similar(word, topn=1)
            if similar_words:
                replaced_text.append(similar_words[0][0])  # 取相近詞中的第一個詞替換原詞
            else:
                replaced_text.append(word)
        else:
            replaced_text.append(word)
    return ' '.join(replaced_text)



In [4]:
try_text = "login issue verified user details employee manager name checked user name reset password advised user login check caller confirmed able login issue resolved"

replaced_texts = replace_random_words(try_text, model)
replaced_texts

'login issue verified advised details employee using name checked user name reset password advised user login check caller confirmed able login issue able'

In [ ]:
# 對每個文本進行詞彙替換
replaced_texts = [replace_random_words(text, model) for text in raw_data['TEXT']]

# 創建替換後的新資料集
replaced_data = raw_data.copy()
replaced_data.drop(columns=['TEXT'])
replaced_data['TEXT'] = replaced_texts

# 保存新資料集
replaced_data.to_csv('data_augmentation' + '_random_' + str(random_words) + '_words' + '.csv', index=False)